In [31]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
sales_df = pd.read_excel("data/Sales_power_oil_agg.xlsx")
print('shape: ',sales_df.shape)
print('column names: ',sales_df.columns)

shape:  (437, 4)
column names:  Index(['Material name', 'Date', 'REGIO', 'Sum of ZIG'], dtype='object')


In [3]:
sales_df = sales_df.rename(columns={'REGIO':'Region','Sum of ZIG':'Qty'})
sales_df.head()

,Material name,Date,Region,Qty
0,BOTTLE-POWER OIL 1.4LX8BOTTLE,2023-05-11,ED,4
1,BOTTLE-POWER OIL 1.4LX8BOTTLE,2023-05-11,FC,3
2,BOTTLE-POWER OIL 1.4LX8BOTTLE,2023-05-11,LA,6
3,BOTTLE-POWER OIL 1.4LX8BOTTLE,2023-05-11,OY,2
4,BOTTLE-POWER OIL 1.4LX8BOTTLE,2023-05-11,RI,2


In [4]:
uniq_materials = sales_df['Material name'].unique()
uniq_regions = sales_df['Region'].unique()
print('Unique SKUs: ',len(uniq_materials))
print(uniq_materials)
print()
print('Unique Regions: ',len(uniq_regions))
print(uniq_regions)

Unique SKUs:  8
['BOTTLE-POWER OIL 1.4LX8BOTTLE' 'BOTTLE-POWER OIL 2.6LX8BOTTLE'
 'BOTTLE-POWER OIL 3LX6BOTTLE' 'BOTTLE-POWER OIL 750MLX12BOTTLE'
 'SACHET-POWER OIL 110MLX40SACHET' 'SACHET-POWER OIL 300MLX24SACHET'
 'SACHET-POWER OIL 45MLX96SACHET' 'SACHET-POWER OIL 50MLX84SACHET']

Unique Regions:  22
['ED' 'FC' 'LA' 'OY' 'RI' 'KN' 'DE' 'IM' 'EN' 'AK' 'EK' 'AB' 'AN' 'BE'
 'BY' 'CR' 'EB' 'ON' 'AD' 'OG' 'KW' 'KO']


In [5]:
sales_df['Date'] = pd.to_datetime(sales_df['Date'])
print(sales_df['Date'].min())
print(sales_df['Date'].max())

2023-05-11 00:00:00
2024-01-10 00:00:00


In [6]:
lengths = {}
for material in uniq_materials:
    for region in uniq_regions:
        filtered_df = sales_df[(sales_df['Material name'] == material) & (sales_df['Region'] == region)]
        length = len(filtered_df)
        lengths[(material, region)] = length

print('dict length: ',len(lengths))
valid_keys = {k: v for k, v in lengths.items() if v}
print('valid dict length: ',len(valid_keys))

dict length:  176
valid dict length:  112


Sales Volume

In [40]:
#Total Sales Volume over the period
total_sales_sku = sales_df[['Material name','Qty']]
total_sales_sku = total_sales_sku.groupby(['Material name']).agg({'Qty':'sum'}).sort_values(by=['Qty'],ascending=False).reset_index()
fig = px.bar(total_sales_sku, x='Material name', y='Qty',title='Total Sales Volume by Material Name')
fig.show()

In [46]:
#Total sales volume by region
total_sales_region = sales_df.groupby(['Region']).agg({'Qty': 'sum'}).sort_values(by='Qty',ascending=False).reset_index()
fig2 = px.bar(total_sales_region, x='Region', y='Qty', title='Total Sales Volume by Region')
fig2.show()


In [56]:
#Region-wise volume sales for each brand
region_wise_sales = sales_df.groupby(['Material name', 'Region']).agg({'Qty':'sum'}).reset_index()


In [ ]:
#Month-wise volume sales


In [65]:
fig_interactive = go.Figure()
for material in uniq_materials:
    material_data = region_wise_sales[region_wise_sales['Material name'] == material]
    fig_interactive.add_trace(
        go.Bar(x=material_data['Region'], y=material_data['Qty'], name=material)
    )

initial_visibility = [True] + [False] * (len(uniq_materials) - 1)
for i, trace in enumerate(fig_interactive.data):
    trace.visible = initial_visibility[i]

# Update layout for dropdowns at the top
fig_interactive.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(label=material,
                     method='update',
                     args=[{'visible': [material == m for m in uniq_materials]}])
                           #{'title': f'Region-wise Volume Sale for {material}'}])
                for material in uniq_materials
            ],
            direction='down',
            showactive=True,
            x=0.5,
            xanchor='center',
            y=1.15,
            yanchor='top'
        )
    ],
    #title='Region-wise Volume Sale for Each Brand',
    height=600,
    width=1000,
    margin=dict(t=100)  # Adjust the top margin to make space for the dropdown
)

fig_interactive.show()


In [71]:
sales_df.groupby(['month_year']).agg({'Qty': 'sum'}).sort_values(by='Qty',ascending=False).reset_index()

,month_year,Qty
0,2024-01,1524170
1,2023-12,438302
2,2023-08,168850
3,2023-11,128781
4,2023-07,15622
5,2023-06,8937
6,2023-05,1228


In [75]:
#Monthly sales volume
sales_df['month_year'] = sales_df['Date'].dt.to_period('M')
total_sales_month = sales_df.groupby(['month_year']).agg({'Qty': 'sum'}).sort_values(by='Qty',ascending=False).reset_index()
total_sales_month['month_year'] = total_sales_month['month_year'].astype(str)
fig3 = px.bar(total_sales_month, x='month_year', y='Qty', title='Total Sales Volume - Month-wise')
fig3.show()